In [1]:
import ast
import math
import numpy as np
import pandas as pd

from itertools import combinations_with_replacement
from tqdm import tqdm
from constants import rares, replace

In [2]:
## Helper Functions OUTSIDE Pasture Setup
def sort_dict(d):
    return {k: v for k, v in reversed(sorted(d.items(), key=lambda item: item[1]))}

In [6]:
class PastureSetup:
    def __init__(self, avg_leavings, max_leavings, pasture_size=20, food_bonus=0.5, debug=False):
        # Defaults
        self.pasture_size = pasture_size
        self.food_bonus = food_bonus
        self.debug = debug
        
        self.avg_leavings = avg_leavings
        self.max_leavings = max_leavings
        
        # DataFrame and Dict Setup (DO NOT CHANGE)
        self.animal_df = pd.read_csv('animals.csv')
        self.animal_dict = dict()
        
        for _, r in self.animal_df.iterrows():
            self.animal_dict[r['Animal']] = [r['Leaving 1'], r['Leaving 2']]
        
    def calculate_buffer(self, buffer_size):
        """
        Calculates Buffer Leavings Usage given Buffer Size
        
        Manual Input
        | buffer_size - How much of a buffer between Avg (0.5) and Max (1) Leavings Usage
        Default Input
        | avg_leavings - Average Leavings Usage
        | max_leavings - Maximum Leavings Usage
        Output
        | buffer_leavings - Buffered Leavings Usage
        """
        mult = buffer_size / 0.5 - 1
        
        buffer_leavings = dict()
        
        for key in self.avg_leavings.keys():
            # Calculate Weekly Leaving Amounts
            buffer_amount = self.avg_leavings[key] + mult * (self.max_leavings[key] - self.avg_leavings[key])
            
            # Calculate Daily Leavings from Weekly, Round, and add to Buffer Leavings Dict
            buffer_leavings[key] = round(buffer_amount / 7,2)
        
        return sort_dict(buffer_leavings)
    
    def calculate_baselines(self, leavings):
        """
        Calculates Base Yield and Bonus Yield given Buffer Leavings Usage
        
        Manual Input
        | leavings - Total Leavings Needed from Animals
        Default Input
        | food_used - Food Used (Currently Hardcoded to Greenfood)
        | pasture_size - Maximum Size of Pasture
        Output
        | base_yield - Base Yield Needed
        | bonus_yield - Bonus Yield Needed
        """
        total_leavings = round(sum(leavings.values()),2)
        estimated_animals = np.floor(total_leavings/1.5)
        estimated_bonus = np.ceil(total_leavings - estimated_animals)

        # If Estimated bonus is greater than estimated animals, then we need to bump up estimated animals
        if estimated_animals < estimated_bonus * 2:
            estimated_animals = estimated_bonus * 2
        
        # Now we can auto-calculate baselines
        base_yield = dict.fromkeys(list(leavings.keys()), 0)
        bonus_yield = dict.fromkeys(list(leavings.keys()), 0)

        t = estimated_animals

        # Genearte Baseline & Bonus from estimated animals needed
        while(t > 0):
            # In case we get over the maximum pasture_size, force leave
            if sum(base_yield.values()) == self.pasture_size:
                break
            
            prev_t = t
            
            for key in leavings.keys():
                if t == 0:
                    break
                if leavings[key] - base_yield[key] > 1.5:
                    t -= 1
                    base_yield[key] += 1

                # Ensure Bonus is always updated
                bonus_yield[key] = max(0, round(leavings[key] - base_yield[key],2))

            # If t doesn't change, find highest bonus leaving and add 1 to the guaranteed
            if prev_t == t:
                max_bonus = ('',0)
                for key in bonus_yield.keys():
                    if bonus_yield[key] > max_bonus[1]:
                        max_bonus = (key, bonus_yield[key])

                max_leaving = max_bonus[0]
                base_yield[max_leaving] += 1
                bonus_yield[max_leaving] = max(0, round(leavings[max_leaving] - base_yield[max_leaving],2))
                t -= 1
                
        # Sanity check in-case
        # Calculates Animals Needed given Food to make sure Bonus makes sense.
        bonus_total = 0
        for val in bonus_yield.values():
            temp = val
            while(temp > 0):
                temp -= self.food_bonus
                bonus_total += 1
                
        if bonus_total > sum(base_yield.values()):
            print('''
            --FATAL ERROR--
            Bonus > Baseline during the sanity check.
            ''')
        if self.debug:
            print('Baseline: {} | Bonus: {}'.format(sum(base_yield.values()), bonus_total))

        return base_yield, bonus_yield
    
    def generate_combinations(self, base_yield):
        """
        Generates all Combinations given Base Yield
        
        Manual Input
        | base_yield - Baseline Yield needed from animals
        Default Input
        | animal_df - Animals Dataframe
        Output
        | combs - All Combinations that satisfy base_yield
        | num_combs - Total Number of Combinations
        """
        def cleanup_rares(l):
            """Cleanes up Rares in Combinations List to ensure Commons are priorities over Rares when possible"""
            cleaned_l = []
            for sub_l in l:
                add = True
                uniques, counts = np.unique(sub_l, return_counts=True)
                for u, c in zip(uniques, counts):
                    if u in rares and c > 1:
                        add=False
                if add:
                    cleaned_l.append(sub_l)
            return cleaned_l
        
        def condense_types(l):
            """Condenses Animals into Groups that share the same Leaving Combo"""
            cleaned_l = []
            for sub_l in l:
                new_l = [replace[a] if a in replace else a for a in sub_l]
                if new_l not in cleaned_l:
                    cleaned_l.append(new_l)
            return cleaned_l
        
        
        combs = dict()
        num_combs = 0
        for k in base_yield.keys():
            temp = self.animal_df.loc[self.animal_df['Leaving 1'] == k]
            if base_yield[k] > 1:
                l = list(combinations_with_replacement(temp['Animal'], base_yield[k]))
                l = cleanup_rares(l)
            else:
                l = [[a] for a in temp['Animal']]
            
            l = condense_types(l)
            
            if num_combs == 0:
                num_combs = len(l)
            elif num_combs > 1:
                num_combs *= len(l)
            combs[k] = l

        return combs, num_combs
    
    def evaluate_combinations(self, combs, num_combs, bonus_yield, buffer):
        """
        Evaluates all Combinations given Bonus Yield
        
        Manual Input
        | combs - All Possible Combinations
        | num_combs - Number of all Possible Combinations
        | bonus_yield - Bonus Yield needed from animals
        |               Base Yield is covered in generate_combinations()
        | buffer - buffer amount for results.csv naming purposes
        Output
        | results_df - DataFrame containing all Valid Results
        """
        results = {
            'Valid':0,
            'Not Enough Bonuses':0
        }

        results_df = pd.DataFrame(columns = ['Animals', 'Leavings', 'Rarity_Value', 'Num_Uniques'])

        with tqdm(total=num_combs) as pbar:
            for c1 in combs['Carapace']:
                for c2 in combs['Fur']:
                    for c3 in combs['Fang']:
                        for c4 in combs['Fleece']:
                            for c5 in combs['Claw']:
                                for c6 in combs['Milk']:
                                    for c7 in combs['Horn']:
                                        for c8 in combs['Feather']:
                                            for c9 in combs['Egg']:
                                                comb_list = c1 + c2 + c3 + c4 + c5 + c6 + c7 + c8 + c9
                                                total_leavings, res = self.evaluate_pasture(comb_list, bonus_yield)
                                                results[res] += 1

                                                if res == 'Valid':
                                                    rarity = round(self.rarity_value(comb_list),2)
                                                    size =  len(np.unique(comb_list))
                                                    results_df.loc[len(results_df)] = [comb_list, total_leavings, rarity, size]

                                                pbar.update(1)
        if self.debug:
            print(results)
        
        if results['Valid'] > 0:
            results_df.to_csv(f'results/Buffer-{buffer}-results.csv', index=False)
            print(f'Results saved to results/Buffer-{buffer}-results.csv')
        else:
            print('No Combinations satisfy Base Yield and Bonus Yield')
        
        return results_df
    
    def evaluate_pasture(self, comb, bonus_yield):
        """
        Evaluates a Combination given Bonus Yield
        
        Manual Input
        | comb - One Possible Combination
        | bonus_yield - Bonus Yield needed from animals
        Output
        | total_leavings - Total Leavings generated on average from this combination
        | res - String result of "Valid" or "Invalid"
        """
        guaranteed, bonus = self.get_leavings(comb)
        g_labels, g_num = np.unique(guaranteed, return_counts=True)
        b_labels, b_num = np.unique(bonus, return_counts=True)
        b_num = np.multiply(b_num, self.food_bonus)

        # Get total
        g_total = {k:v for k, v in zip(g_labels, g_num)}
        b_total = {k:v for k, v in zip(b_labels, b_num)}
        total_leavings = dict()

        for i, _ in enumerate(g_labels):
            total_leavings[g_labels[i]] = g_num[i] 
        for i, _ in enumerate(b_labels):
            total_leavings[b_labels[i]] += round(b_num[i],1)

        total_leavings = sort_dict(total_leavings)

        # We already guarantee the base yield when generating the initial lists. We just need to calculate the bonus yield
        # If any of our totals are less than the bonus yield threshold for any leaving, break
        for key in total_leavings.keys():
            # If we're expecting a bonus and the list doesn't offer one, return 1
            if key not in b_total.keys() and key in bonus_yield.keys():
                return total_leavings, 'Not Enough Bonuses'
            # If we're expecting a bonus and the list doesn't offer enough, return 2
            if b_total[key] < bonus_yield[key]:
                return total_leavings, 'Not Enough Bonuses'

        return total_leavings, 'Valid'
    
    ##
    # Helper Functions
    ##
    
    def get_leavings(self, comb):
        """Takes Animal List and Returns what Leavings it generates (Guaranteed & Bonus)"""
        leavings = []
        for a in comb:
            if '/' in a:
                a = a.split('/')[0]
            leavings.append(self.animal_dict[a])

        guaranteed, bonus = zip(*leavings)
        return guaranteed, bonus
    
    def rarity_value(self, a_list):
        """Calculates the Rarity Value of an Animal List, which represents how many rares exist in a given list"""
        rarity_dict = {
            'C':0,
            'T':1,
            'W':2,
            'TW':3
        }

        total = 0

        for a in a_list:
            spl = a.split('/')
            if len(spl) > 1:
                rarity1 = list(self.animal_df.loc[self.animal_df['Animal'] == spl[0]]['Type'])[0]
                rarity2 = list(self.animal_df.loc[self.animal_df['Animal'] == spl[1]]['Type'])[0]

                rarity1 = rarity_dict[rarity1]
                rarity2 = rarity_dict[rarity2]
                total += min(rarity1, rarity2)
            else:
                rarity1 = list(self.animal_df.loc[self.animal_df['Animal'] == spl[0]]['Type'])[0]
                rarity1 = rarity_dict[rarity1]
                total += rarity1
        return total/len(a_list)
    
    def print_results(self, a_list, total_leavings, rarity_val):
        """Prints the Results of a Given List, including Leavings Gained"""
        g, b = dict(), dict()

        print('--Pasture | Rarity: {}--'.format(rarity_val))
        for a in sorted(a_list):
            a0 = a.split('/')[0]

            l1 = self.animal_dict[a0][0]
            l2 = self.animal_dict[a0][1]

            print(f'| {a} - ({l1}/{l2})')

            if l1 not in g.keys():
                g[l1] = 1
            else:
                g[l1] += 1

            if l2 not in b.keys():
                b[l2] = 1
            else:
                b[l2] += 1

        print('--Avg Leavings per Day')
        for t in total_leavings.keys():
            print(f'| {t}-{round(total_leavings[t],1)} | Guaranteed: {g[t]} | Bonus: {b[t]}')

# Test

In [7]:
max_weekly = {
    'Carapace': 58,
    'Claw': 45,
    'Egg': 23,
    'Fang': 51,
    'Feather': 20,
    'Fleece': 36,
    'Fur': 40,
    'Horn': 18,
    'Milk': 37,
}

avg_weekly = {
    'Carapace': 11.7,
    'Claw': 10.1,
    'Egg': 3.7,
    'Fang': 10.9,
    'Feather': 2.4,
    'Fleece': 8.2,
    'Fur': 8.3,
    'Horn': 3.4,
    'Milk': 7.9,
}
setup = PastureSetup(avg_weekly, max_weekly, debug=True)

In [8]:
buffer = .5
leavings = setup.calculate_buffer(buffer)
g,b = setup.calculate_baselines(leavings)
combs, num_combs = setup.generate_combinations(g)
results50 = setup.evaluate_combinations(combs, num_combs, b, buffer)

Baseline: 8 | Bonus: 8


  1%|▊                                                                           | 647/64800 [00:00<00:09, 6429.40it/s]C:\Users\Jarrod\miniconda3\envs\general\lib\site-packages\pandas\core\dtypes\cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)
100%|██████████████████████████████████████████████████████████████████████████| 64800/64800 [00:10<00:00, 6227.42it/s]

{'Valid': 81, 'Not Enough Bonuses': 64719}
Results saved to Buffer-0.5-results.csv


In [9]:
buffer = .65
leavings = setup.calculate_buffer(buffer)
g,b = setup.calculate_baselines(leavings)
combs, num_combs = setup.generate_combinations(g)
results65 = setup.evaluate_combinations(combs, num_combs, b, buffer)

Baseline: 16 | Bonus: 14


100%|██████████████████████████████████████████████████████████████████████| 1179360/1179360 [03:27<00:00, 5686.68it/s]

{'Valid': 1311, 'Not Enough Bonuses': 1178049}
Results saved to Buffer-0.65-results.csv


In [10]:
buffer = .75
leavings = setup.calculate_buffer(buffer)
g,b = setup.calculate_baselines(leavings)
combs, num_combs = setup.generate_combinations(g)
results75 = setup.evaluate_combinations(combs, num_combs, b, buffer)

Baseline: 20 | Bonus: 20


100%|██████████████████████████████████████████████████████████████████████| 2488320/2488320 [07:00<00:00, 5910.94it/s]

{'Valid': 6, 'Not Enough Bonuses': 2488314}
Results saved to Buffer-0.75-results.csv


# Manual Evaluations

In [21]:
# 50%
# Minimum Rares
results50_temp = results50.loc[(results50['Rarity_Value'] == min(results50['Rarity_Value']))]
results50_temp.head(10)

,Animals,Leavings,Rarity_Value,Num_Uniques
29,"[Beachcomb/Glyptodon Pup, Yellow Coblyn/Morbol...","{'Carapace': 3.0, 'Fang': 2.5, 'Egg': 2.0, 'Mi...",0.36,10
33,"[Beachcomb/Glyptodon Pup, Yellow Coblyn/Morbol...","{'Fang': 2.5, 'Carapace': 2.5, 'Fur': 2.0, 'Eg...",0.36,11
37,"[Beachcomb/Glyptodon Pup, Yellow Coblyn/Morbol...","{'Fang': 2.5, 'Carapace': 2.5, 'Horn': 2.0, 'E...",0.36,11


In [22]:
for _, r in results50_temp.iterrows():
    setup.print_results(r['Animals'], r['Leavings'], r['Rarity_Value'])
    print('')

--Pasture | Rarity: 0.36--
| Apkallu - (Fleece/Egg)
| Aurochs/Island Nanny - (Milk/Horn)
| Beachcomb/Glyptodon Pup - (Carapace/Claw)
| Blue Back/Dodo - (Egg/Feather)
| Coblyn - (Fang/Carapace)
| Coblyn - (Fang/Carapace)
| Dodo of Paradise/Gold Back/Alkonost - (Feather/Egg)
| Ground Squirrel/Opo-Opo - (Claw/Fur)
| Island Billy - (Horn/Fleece)
| Island Doe - (Fur/Milk)
| Yellow Coblyn/Morbol - (Carapace/Fang)
--Avg Leavings per Day
| Carapace-3.0 | Guaranteed: 2 | Bonus: 2
| Fang-2.5 | Guaranteed: 2 | Bonus: 1
| Egg-2.0 | Guaranteed: 1 | Bonus: 2
| Milk-1.5 | Guaranteed: 1 | Bonus: 1
| Horn-1.5 | Guaranteed: 1 | Bonus: 1
| Fur-1.5 | Guaranteed: 1 | Bonus: 1
| Fleece-1.5 | Guaranteed: 1 | Bonus: 1
| Feather-1.5 | Guaranteed: 1 | Bonus: 1
| Claw-1.5 | Guaranteed: 1 | Bonus: 1

--Pasture | Rarity: 0.36--
| Apkallu - (Fleece/Egg)
| Aurochs/Island Nanny - (Milk/Horn)
| Beachcomb/Glyptodon Pup - (Carapace/Claw)
| Blue Back/Dodo - (Egg/Feather)
| Coblyn - (Fang/Carapace)
| Dodo of Paradise/Gold

In [26]:
# 65%
results65_temp = results65.loc[(results65['Rarity_Value'] == max(results65['Rarity_Value']))]
results65_temp.head(10)

,Animals,Leavings,Rarity_Value,Num_Uniques
1139,"[Beachcomb/Glyptodon Pup, Yellow Coblyn/Morbol...","{'Fang': 4.5, 'Carapace': 4.0, 'Fur': 3.0, 'Fl...",1.41,13


In [27]:
for _, r in results65_temp.iterrows():
    setup.print_results(r['Animals'], r['Leavings'], r['Rarity_Value'])
    print('')

--Pasture | Rarity: 1.41--
| Alligator - (Claw/Fang)
| Aurochs/Island Nanny - (Milk/Horn)
| Beachcomb/Glyptodon Pup - (Carapace/Claw)
| Blue Back/Dodo - (Egg/Feather)
| Chocobo - (Fur/Feather)
| Chocobo - (Fur/Feather)
| Coblyn - (Fang/Carapace)
| Coblyn - (Fang/Carapace)
| Dodo of Paradise/Gold Back/Alkonost - (Feather/Egg)
| Goobue - (Fang/Claw)
| Grand Buffalo - (Horn/Milk)
| Ornery Karakul - (Milk/Fleece)
| Paissa - (Claw/Fleece)
| Twinklefleece/Funguar - (Fleece/Fur)
| Twinklefleece/Funguar - (Fleece/Fur)
| Yellow Coblyn/Morbol - (Carapace/Fang)
| Yellow Coblyn/Morbol - (Carapace/Fang)
--Avg Leavings per Day
| Fang-4.5 | Guaranteed: 3 | Bonus: 3
| Carapace-4.0 | Guaranteed: 3 | Bonus: 2
| Fur-3.0 | Guaranteed: 2 | Bonus: 2
| Fleece-3.0 | Guaranteed: 2 | Bonus: 2
| Claw-3.0 | Guaranteed: 2 | Bonus: 2
| Milk-2.5 | Guaranteed: 2 | Bonus: 1
| Feather-2.5 | Guaranteed: 1 | Bonus: 3
| Horn-1.5 | Guaranteed: 1 | Bonus: 1
| Egg-1.5 | Guaranteed: 1 | Bonus: 1



In [30]:
# 75%
results75_temp = results75.loc[(results75['Rarity_Value'] == max(results75['Rarity_Value']))]
results75_temp.head(10)

,Animals,Leavings,Rarity_Value,Num_Uniques
1,"[Beachcomb/Glyptodon Pup, Beachcomb/Glyptodon ...","{'Carapace': 5.0, 'Fang': 4.5, 'Claw': 4.0, 'M...",0.85,17
2,"[Beachcomb/Glyptodon Pup, Beachcomb/Glyptodon ...","{'Carapace': 5.0, 'Fang': 4.5, 'Claw': 4.0, 'M...",0.85,17


In [31]:
for _, r in results75_temp.iterrows():
    setup.print_results(r['Animals'], r['Leavings'], r['Rarity_Value'])
    print('')

--Pasture | Rarity: 0.85--
| Alligator - (Claw/Fang)
| Apkallu - (Fleece/Egg)
| Apkallu of Paradise - (Egg/Fleece)
| Aurochs/Island Nanny - (Milk/Horn)
| Beachcomb/Glyptodon Pup - (Carapace/Claw)
| Beachcomb/Glyptodon Pup - (Carapace/Claw)
| Chocobo - (Fur/Feather)
| Chocobo - (Fur/Feather)
| Coblyn - (Fang/Carapace)
| Dodo of Paradise/Gold Back/Alkonost - (Feather/Egg)
| Glyptodon - (Claw/Carapace)
| Grand Buffalo - (Horn/Milk)
| Island Doe - (Fur/Milk)
| Lost Lamb - (Fleece/Milk)
| Ornery Karakul - (Milk/Fleece)
| Paissa - (Claw/Fleece)
| Tiger/Quartz Spriggan/Weird Spriggan - (Fang/Fur)
| Wild Boar - (Fang/Horn)
| Yellow Coblyn/Morbol - (Carapace/Fang)
| Yellow Coblyn/Morbol - (Carapace/Fang)
--Avg Leavings per Day
| Carapace-5.0 | Guaranteed: 4 | Bonus: 2
| Fang-4.5 | Guaranteed: 3 | Bonus: 3
| Claw-4.0 | Guaranteed: 3 | Bonus: 2
| Milk-3.5 | Guaranteed: 2 | Bonus: 3
| Fur-3.5 | Guaranteed: 3 | Bonus: 1
| Fleece-3.5 | Guaranteed: 2 | Bonus: 3
| Horn-2.0 | Guaranteed: 1 | Bonus: 2
|